# NPG
---

## natural gradient
* 신경망 모델을 이루는 파라미터의 매니폴드가 업데이트 될 때
* 기존에는 대부분 그냥 유클리드 공간에서 기울기를 구하고 경사하강을 적용하여 업데이트
* 그런데 매니폴드 입장에서는 부당하다고 볼 수 있음
* 매니폴드는 곡면이나 더 복잡한 형태로 형성되어 있을 수 있는데 그걸 무시하고 유클리드 공간에서 업데이트가 되어버리면 불안정하고 급격하게 바뀌는 결과 초래
* 그래서 리만 공간에서 자연스러운 파라미터 업데이트하는 학습 갱신 방법 제시
* natural 은 covariant
* policy 를 이루는 파라미터들이 업데이트 될 때 매니폴드를 따라서 안정적으로 업데이트 하는 목적
* RL 자체가 상호작용이 있어 민감하기 때문에, 다른 분야보다 안정적 업데이트에 더 힘을 써야함.
* 결국 파라미터를 어떻게 진짜 steepest 한 방향으로 업데이트 할 것인가의 문제

## 추가 설명
* 원래 고차원 파라미터 공간에서 업데이트 하는 것이 기본
* 그런데 특정 파라미터만이 최적화에 크리티컬한 영향을 미침
* 이러한 파라미터들이 이루는 파라미터 매니폴드 라는 서브 스페이스를 생각할 수 있음
* 이 공간을 리만 곡면, 리만 매니폴드라고 함
* 이 곡면을 따라 파라미터의 경사 하강을 수행하면 더 중요한 파라미터 방향으로 업데이트가 되며 더 안정적이고 빠른 최적화가 가능함
* 현재 파라미터 위치에서 국소적으로 리만 곡면을 추정하고 그 곡면을 따라 움직일 것
* 근데 그 곡면은 양의 정부호 (일부 헤시안, FIM 등) 를 곱해서 구함
* 그 행렬을 곱하면 자연스럽게 중요한 파라미터들로 이루어진 국소 곡면을 알 수 있음
* 만약 그 행렬이 항등 행렬이면 곡면을 무시해버리고 일반적 경사하강처럼 모든 파라미터를 공평하게 영향을 줌

In [ ]:
import numpy as np
from utils.utils import *
from hparams import HyperParams as hp


def get_returns(rewards, masks):
    rewards = torch.Tensor(rewards)
    masks = torch.Tensor(masks)
    returns = torch.zeros_like(rewards)

    running_returns = 0

    for t in reversed(range(0, len(rewards))):
        running_returns = rewards[t] + hp.gamma * running_returns * masks[t]
        returns[t] = running_returns

    returns = (returns - returns.mean()) / returns.std()
    return returns


def get_loss(actor, returns, states, actions):
    mu, std, logstd = actor(torch.Tensor(states))
    log_policy = log_density(torch.Tensor(actions), mu, std, logstd)
    returns = returns.unsqueeze(1)

    objective = returns * log_policy
    objective = objective.mean()
    return objective


def train_critic(critic, states, returns, critic_optim):
    criterion = torch.nn.MSELoss()
    n = len(states)
    arr = np.arange(n)

    for epoch in range(5):
        np.random.shuffle(arr)

        for i in range(n // hp.batch_size):
            batch_index = arr[hp.batch_size * i: hp.batch_size * (i + 1)]
            batch_index = torch.LongTensor(batch_index)
            inputs = torch.Tensor(states)[batch_index]
            target = returns.unsqueeze(1)[batch_index]

            values = critic(inputs)
            loss = criterion(values, target)
            critic_optim.zero_grad()
            loss.backward()
            critic_optim.step()


def fisher_vector_product(actor, states, p):
    p.detach()
    kl = kl_divergence(new_actor=actor, old_actor=actor, states=states)
    kl = kl.mean()
    kl_grad = torch.autograd.grad(kl, actor.parameters(), create_graph=True)
    kl_grad = flat_grad(kl_grad)  # check kl_grad == 0

    kl_grad_p = (kl_grad * p).sum()
    kl_hessian_p = torch.autograd.grad(kl_grad_p, actor.parameters())
    kl_hessian_p = flat_hessian(kl_hessian_p)

    return kl_hessian_p + 0.1 * p


# from openai baseline code
# https://github.com/openai/baselines/blob/master/baselines/common/cg.py
def conjugate_gradient(actor, states, b, nsteps, residual_tol=1e-10):
    x = torch.zeros(b.size())
    r = b.clone()
    p = b.clone()
    rdotr = torch.dot(r, r)
    for i in range(nsteps):
        _Avp = fisher_vector_product(actor, states, p)
        alpha = rdotr / torch.dot(p, _Avp)
        x += alpha * p
        r -= alpha * _Avp
        new_rdotr = torch.dot(r, r)
        betta = new_rdotr / rdotr
        p = r + betta * p
        rdotr = new_rdotr
        if rdotr < residual_tol:
            break
    return x


def train_model(actor, critic, memory, actor_optim, critic_optim):
    memory = np.array(memory)
    states = np.vstack(memory[:, 0])
    actions = list(memory[:, 1])
    rewards = list(memory[:, 2])
    masks = list(memory[:, 3])

    # ----------------------------
    # step 1: get returns
    returns = get_returns(rewards, masks)

    # ----------------------------
    # step 2: train critic several steps with respect to returns
    train_critic(critic, states, returns, critic_optim)

    # ----------------------------
    # step 3: get gradient of loss and hessian of kl
    loss = get_loss(actor, returns, states, actions)
    loss_grad = torch.autograd.grad(loss, actor.parameters())
    loss_grad = flat_grad(loss_grad)
    step_dir = conjugate_gradient(actor, states, loss_grad.data, nsteps=10)

    # ----------------------------
    # step 4: get step direction and step size and update actor
    params = flat_params(actor)
    new_params = params + 0.5 * step_dir
    update_model(actor, new_params)

